In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=cd0bbd8cb40c182379c89d9b198d1a1cdcc5a088e2ba8c7329fccbcd98f409cd
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-mag0qg2c/kobert-tokenizer_8b4260db27cd4d0db36fd8d17ac1a118
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-mag0qg2c/kobert-tokenizer_8b4260db27cd4d0db36fd8d17ac1a118
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=2b87455e7d33cdc128bcfabc2e074cf3a9eeff81cd471262f92a92299cfe7179
  Stored in directory: /tmp/pip-ephem-wheel-cache-mw8y94hk/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# GPU 사용 시
device = torch.device("cuda:0")

In [7]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
train1 = pd.read_csv('/content/train_data.csv')
test1 = pd.read_csv('/content/test_data.csv')

In [9]:
train1.head()

,title,score
0,네이버 방탄소년단 이야기로 해외웹툰 공략 네이버 vs 카카오 새로운 격전지는 디지털...,0
1,네이버 메타버스 쌍두마차 제페토 아크버스 진격 앞으로 플랫폼 이어 메타버스 NFT까...,0
2,본인확인기관 심사기준 완화됐지만 네이버 카카오 부적합 항목 해소해야 나만의 금융비서...,0
3,네이버 카카오 등 플랫폼 자사 상품 우대하면 불법 외국인도 기관도 팔아치우는 네이버...,1
4,구글 네이버 등 플랫폼 자사우대 끼워팔기 금지 커넥티드 카 소프트웨어 깊은 매력 기...,1


In [10]:
test1.head()

,title,score
0,대어 LG엔솔 잡은 KB 올 IPO 주관 1위 꿰찼다,0
1,디즈니 같은 회사 꿈꾸던 김정주 어벤져스 감독 제작사에 6000억 시총 10위 탈환...,0
2,연초부터 흔들린 게임 주식부자 순위 지각변동 디즈니처럼 게임회사들 영화 테마파크로 ...,0
3,돈 버는 게임 재밌는 게임 P2E 열풍에 엇갈린 생존 게임 12일 배틀그라운드 무료...,1
4,현대제철 양재동 떠난다,0


In [11]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   907 non-null    object
 1   score   907 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.3+ KB


In [12]:
train1.groupby('score').count()

,title
score,
0,350
1,292
2,265


In [13]:
test1.groupby('score').count()

,title
score,
0,92
1,64
2,71


In [14]:
from sklearn.model_selection import train_test_split
merged = pd.concat([test1, train1], axis = 0)
train, test = train_test_split(merged, test_size=0.2, random_state=123)

In [15]:
merged.head()

,title,score
0,대어 LG엔솔 잡은 KB 올 IPO 주관 1위 꿰찼다,0
1,디즈니 같은 회사 꿈꾸던 김정주 어벤져스 감독 제작사에 6000억 시총 10위 탈환...,0
2,연초부터 흔들린 게임 주식부자 순위 지각변동 디즈니처럼 게임회사들 영화 테마파크로 ...,0
3,돈 버는 게임 재밌는 게임 P2E 열풍에 엇갈린 생존 게임 12일 배틀그라운드 무료...,1
4,현대제철 양재동 떠난다,0


In [16]:
class BERTSentenceTransform:
    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab 

    def __call__(self, line):
       
        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
           
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
          
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        valid_length = len(input_ids)

        if self._pad:
          
            padding_length = self._max_seq_length - valid_length
           
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer ,vocab, max_len,
                 pad, pair):
   
        #transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab = vocab, pad=pad, pair=pair)
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab = vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [18]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
#tok = tokenizer.tokenize

data_train = BERTDataset(train.values, 0, 1, tokenizer,  vocab, max_len, True, False)
data_test = BERTDataset(test.values, 0, 1, tokenizer, vocab, max_len, True, False)

In [20]:
data_train[0]

(array([   2, 1237, 2670, 5579, 1971, 3396, 3996, 5550, 1289, 7086, 1837,
        3862, 5761, 4760, 5546, 7788, 4635, 7631, 7078, 2145, 4196, 2143,
        6364, 6198, 3135, 1761, 1618, 6733, 7086,  659,    0, 5808, 7139,
        3761, 7044, 3647, 7591, 6263, 4455, 5965, 4578, 7229, 3481, 5760,
        3761, 7491, 7848, 5211, 2086, 6779, 5760, 1117,  517, 5345, 7350,
        6305, 7334,  517, 6896, 2284, 4458, 6084, 1088,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [21]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [22]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [23]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [24]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [26]:
train_history = []
test_history = []
loss_history = []
best_accuracy = 0.0
best_model = None
early_stop = 0

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length.to(device)  # valid_length에도 .to(device) 적용
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        train_batch_acc = calc_accuracy(out, label)  # 배치별 정확도 기록
        train_acc += train_batch_acc
        train_history.append(train_batch_acc.item())  # 배치별 정확도 저장
        loss_history.append(loss.item())

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(
                e+1, batch_id+1, loss.item(), train_batch_acc.item()))

    avg_train_acc = train_acc / len(train_dataloader)  # 평균 정확도 계산
    print("epoch {} train acc {}".format(e+1, avg_train_acc))

    model.eval()

    with torch.no_grad():
        test_acc = 0.0

        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length.to(device)
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)

            test_batch_acc = calc_accuracy(out, label)
            test_acc += test_batch_acc
            test_history.append(test_batch_acc.item())

        avg_test_acc = test_acc / len(test_dataloader)
        print("epoch {} test acc {}".format(e+1, avg_test_acc))
        test_history.append(avg_test_acc.item())

        # 성능이 가장 좋을 때 모델 저장
        if avg_test_acc > best_accuracy:
            best_accuracy = avg_test_acc
            best_model = model.state_dict()
            early_stop = 0

        elif early_stop == 3:
          break

        else:
          early_stop += 1

# 가장 좋은 모델 저장
torch.save(best_model, 'best_model.pth')

  7%|▋         | 1/15 [00:01<00:24,  1.74s/it]

epoch 1 batch id 1 loss 1.1627978086471558 train acc 0.296875


100%|██████████| 15/15 [00:09<00:00,  1.58it/s]


epoch 1 train acc 0.35823863636363634


100%|██████████| 4/4 [00:00<00:00,  4.96it/s]


...

epoch 12 batch id 1 loss 0.4466347098350525 train acc 0.828125


100%|██████████| 15/15 [00:09<00:00,  1.65it/s]


epoch 12 train acc 0.8364583333333333


100%|██████████| 4/4 [00:00<00:00,  4.49it/s]


epoch 12 test acc 0.36428571428571427
